In [1]:
import numpy as np
import pandas as pd

from ols_bootstrap.pairs import PairsBootstrap
from ols_bootstrap.residual import ResidualBootstrap
from ols_bootstrap.wild import WildBootstrap

pd.options.display.float_format = '{:20,.5f}'.format  ### Setting pd to have a numerical precision up to 5 decimal points

In [2]:
df = pd.read_csv('./balance2018.csv')
df = df[df['sales_clean'] != 0]
df = df[['sales_clean', 'tanass_clean', 'tax']]
df = df.dropna(subset=['tanass_clean', 'tax'])

df_scaled = df.applymap(lambda x: np.log(x + 1))

/home/phamv/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (66) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
df_sample = df_scaled.sample(n=10000, replace=False)

Y_data = pd.DataFrame(df_sample.iloc[:, 0])
X_data = pd.DataFrame(df_sample.iloc[:, 1:])

## Default SE on the original OLS is HC3, default CI on bootstrapped parameter is BC. 

That is by default se_type = 'hc3', ci_type = 'bc'.

In [4]:
psb = PairsBootstrap(Y_data, X_data, reps = 1000, se_type='hc3', ci_type = 'bc')  # se_type = 'hc3' and ci_type = 'bc' are default options for these arguments.
psb.fit()

In [5]:
rsb = ResidualBootstrap(Y_data, X_data, reps = 1000)
rsb.fit()

In [6]:
wb_stdn = WildBootstrap(Y_data, X_data, reps = 1000, from_distro = "standard_normal")
wb_stdn.fit()

In [7]:
wb_rad = WildBootstrap(Y_data, X_data, reps = 1000, from_distro = "rademacher")
wb_rad.fit()

In [8]:
wb_mam = WildBootstrap(Y_data, X_data, reps = 1000, from_distro = "mammen")
wb_mam.fit()

In [9]:
psb.summary()

+----------------------------------------------------------------------------------------------------------------------------------+
|                     Pairs Bootstrap results with 10000 obs and 1000 BS reps using HC3 SE-s and 95.00% BC CI                      |
+--------------+------------+------------------+--------+---------------+--------------+--------------+------------------+---------+
|     Var      | OLS Params | Avg of BS Params |  Bias  | OLS Params SE | BS Params SE | % of SE Diff |        CI        | CI Diff |
+--------------+------------+------------------+--------+---------------+--------------+--------------+------------------+---------+
|    const     |   6.2680   |      6.2685      | 0.0004 |     0.0451    |    0.0463    |    -2.65     | [6.1812, 6.3658] |  0.1846 |
+--------------+------------+------------------+--------+---------------+--------------+--------------+------------------+---------+
| tanass_clean |   0.1687   |      0.1689      | 0.0001 |     0.0049 

In [10]:
rsb.summary()

+----------------------------------------------------------------------------------------------------------------------------------+
|                    Residual Bootstrap results with 10000 obs and 1000 BS reps using HC3 SE-s and 95.00% BC CI                    |
+--------------+------------+------------------+--------+---------------+--------------+--------------+------------------+---------+
|     Var      | OLS Params | Avg of BS Params |  Bias  | OLS Params SE | BS Params SE | % of SE Diff |        CI        | CI Diff |
+--------------+------------+------------------+--------+---------------+--------------+--------------+------------------+---------+
|    const     |   6.2680   |      6.2692      | 0.0012 |     0.0451    |    0.0350    |    22.45     | [6.1960, 6.3317] |  0.1357 |
+--------------+------------+------------------+--------+---------------+--------------+--------------+------------------+---------+
| tanass_clean |   0.1687   |      0.1687      | 0.0001 |     0.0049 

In [11]:
wb_stdn.summary()

+----------------------------------------------------------------------------------------------------------------------------------+
|           Wild Bootstrap with Standard Normal results with 10000 obs and 1000 BS reps using HC3 SE-s and 95.00% BC CI            |
+--------------+------------+------------------+--------+---------------+--------------+--------------+------------------+---------+
|     Var      | OLS Params | Avg of BS Params |  Bias  | OLS Params SE | BS Params SE | % of SE Diff |        CI        | CI Diff |
+--------------+------------+------------------+--------+---------------+--------------+--------------+------------------+---------+
|    const     |   6.2680   |      6.2694      | 0.0014 |     0.0451    |    0.0338    |    25.08     | [6.2007, 6.3298] |  0.1291 |
+--------------+------------+------------------+--------+---------------+--------------+--------------+------------------+---------+
| tanass_clean |   0.1687   |      0.1688      | 0.0000 |     0.0049 

In [12]:
wb_rad.summary()

+----------------------------------------------------------------------------------------------------------------------------------+
|              Wild Bootstrap with Rademacher results with 10000 obs and 1000 BS reps using HC3 SE-s and 95.00% BC CI              |
+--------------+------------+------------------+--------+---------------+--------------+--------------+------------------+---------+
|     Var      | OLS Params | Avg of BS Params |  Bias  | OLS Params SE | BS Params SE | % of SE Diff |        CI        | CI Diff |
+--------------+------------+------------------+--------+---------------+--------------+--------------+------------------+---------+
|    const     |   6.2680   |      6.2675      | 0.0006 |     0.0451    |    0.0351    |    22.04     | [6.2017, 6.3398] |  0.1381 |
+--------------+------------+------------------+--------+---------------+--------------+--------------+------------------+---------+
| tanass_clean |   0.1687   |      0.1687      | 0.0000 |     0.0049 

In [13]:
wb_mam.summary()

+----------------------------------------------------------------------------------------------------------------------------------+
|                Wild Bootstrap with Mammen results with 10000 obs and 1000 BS reps using HC3 SE-s and 95.00% BC CI                |
+--------------+------------+------------------+--------+---------------+--------------+--------------+------------------+---------+
|     Var      | OLS Params | Avg of BS Params |  Bias  | OLS Params SE | BS Params SE | % of SE Diff |        CI        | CI Diff |
+--------------+------------+------------------+--------+---------------+--------------+--------------+------------------+---------+
|    const     |   6.2680   |      6.2715      | 0.0034 |     0.0451    |    0.0338    |    24.92     | [6.1981, 6.3311] |  0.1330 |
+--------------+------------+------------------+--------+---------------+--------------+--------------+------------------+---------+
| tanass_clean |   0.1687   |      0.1687      | 0.0000 |     0.0049 

## Some useful methods were implemented (let's use it on wb_mam object of Wild Bootstrap with Mammen)

- The common in the following three methods is that either a string (if one wishes to capture one variable or ci) or 1-D like array can be provided to 'which_var' and/or 'which_ci' (the latter if exists in that object class).

### get_ci() method

#### Vanila version is when only the actual CI was used with all independent variables

In [14]:
wb_mam.get_ci()

bc                     
                              lwb                  upb
const                     6.19809              6.33112
tanass_clean              0.16103              0.17565
tax                       0.50690              0.53288

#### However, any combination of ('bc', 'bca', 'percentile) CI types could be selected and any combinations of independent variables can be chosen with 'which_ci' and 'which_var' optional arguments, respectively.

Please note that if choosing 'bca' the calculation can take a while as it uses jacknife resampling for calculating the acceleration factor

In [15]:
wb_mam.get_ci(which_ci=['bc', 'percentile'], which_var=['tax', 'const'])

bc                                percentile  \
                       lwb                  upb                  lwb   
tax                0.50690              0.53288              0.50520   
const              6.19809              6.33112              6.20779   

                            
                       upb  
tax                0.53095  
const              6.33931

In [16]:
wb_mam.get_ci(which_ci='bca', which_var='tanass_clean')

bca                     
                              lwb                  upb
tanass_clean              0.16117              0.17569

### get_all_se() method

The following SE-s are calculated: 
- bootstrapped - standard error of the bootstrapped parameters
- constant - model-based OLS Standard Errors, that is, constant variance is assumed 
- HC0, HC1, HC2, HC3, HC4, HC4m, HC5 - Heteroskedasticity-Consistent Standard Errors (HCE) using sandwich estimators 

#### Vanila version is when using all indepencdent variables. 

In [17]:
wb_mam.get_all_se()

,bootstrapped,constant,hc0,hc1,hc2,hc3,hc4,hc4m,hc5
const,0.03385,0.03530,0.04506,0.04507,0.04507,0.04508,0.04508,0.04509,0.04508
tanass_clean,0.00370,0.00386,0.00494,0.00494,0.00494,0.00494,0.00494,0.00494,0.00494
tax,0.00662,0.00646,0.00908,0.00908,0.00908,0.00909,0.00909,0.00909,0.00909


#### A subset of indepent variables can be chosen with 'which_var' argument to calculate the above-mentioned 9 SE-s

In [18]:
wb_mam.get_all_se(which_var=['tanass_clean', 'tax'])

,bootstrapped,constant,hc0,hc1,hc2,hc3,hc4,hc4m,hc5
tanass_clean,0.00370,0.00386,0.00494,0.00494,0.00494,0.00494,0.00494,0.00494,0.00494
tax,0.00662,0.00646,0.00908,0.00908,0.00908,0.00909,0.00909,0.00909,0.00909


In [19]:
wb_mam.get_all_se(which_var='tax')

,bootstrapped,constant,hc0,hc1,hc2,hc3,hc4,hc4m,hc5
tax,0.00662,0.00646,0.00908,0.00908,0.00908,0.00909,0.00909,0.00909,0.00909


### get_bootstrap_params() method

#### Vanila version: Returning a dataframe capturing the parameter estimate of ALL each independent variables in each (wild) bootstrap.

In [20]:
wb_mam.get_bootstrap_params()

,const,tanass_clean,tax
0,6.29879,0.16741,0.51085
1,6.28686,0.17039,0.51341
2,6.27839,0.16602,0.52103
3,6.31884,0.17341,0.50034
4,6.24180,0.17666,0.51262
...,...,...,...
995,6.23622,0.17269,0.52039
996,6.25913,0.17086,0.51663
997,6.29334,0.16435,0.52022
998,6.26771,0.16733,0.51924


#### As usual, the desired independent variable can be chosen with 'which_var' argument

In [21]:
wb_mam.get_bootstrap_params(which_var='tax')

,tax
0,0.51085
1,0.51341
2,0.52103
3,0.50034
4,0.51262
...,...
995,0.52039
996,0.51663
997,0.52022
998,0.51924


In [22]:
wb_mam.get_bootstrap_params(which_var=('const', 'tanass_clean'))

,const,tanass_clean
0,6.29879,0.16741
1,6.28686,0.17039
2,6.27839,0.16602
3,6.31884,0.17341
4,6.24180,0.17666
...,...,...
995,6.23622,0.17269
996,6.25913,0.17086
997,6.29334,0.16435
998,6.26771,0.16733


### bp_test() and white_test() methods

#### Vanila version: for bp_test() returning the robust version (proposed by Roger Koenker) of this test when the presence of the heteroscedasticity is examined on the residual of the OLS on the original sample data. For white_test(), there are no optional arguments

In [23]:
wb_mam.bp_test()

(633.0898079721314,
 3.3596568712634616e-138,
 337.8381280779217,
 1.0577598863840659e-142)

In [24]:
wb_mam.white_test()

(1883.0473052848306, 0.0, 463.700491473099, 0.0)

#### The non-robust version of bp_test() can be achieved by setting bp_test(robust = False).

In [25]:
wb_mam.bp_test(robust = False)

(1268.5038009107557,
 3.5310144560585667e-276,
 337.8381280779195,
 1.05775988638599e-142)

At each test, the first value is the calculated F test stats, the second is the 'F p-value' corresponding to the F-stats, the third value is the LM test stats and last one is the 'LM p-value'.

As it can be seen the p-values are 0, so we can rejcet the null hypothesis of homoscedasticity, and accept the alternative hypothesis that the variance of the residuals is heteroscedastic.